# Clustering Crypto

In [79]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [80]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [81]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
file_path = Path("Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.describe()
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [82]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df.drop(["Unnamed: 0"], axis=1, inplace=True)
crypto_df.head(300)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
295,BitLux,X11,False,PoW/PoS,NaN,0
296,FutCoin,X13,True,PoW/PoS,NaN,0
297,TamaGucci,Scrypt,True,PoW/PoS/PoC,NaN,5300000
298,MorpheusCoin,X11,True,PoW,NaN,3400000


In [83]:
# Keep only cryptocurrencies that are trading
crypto_df.drop(crypto_df[crypto_df['IsTrading'] == False].index, inplace=True)
crypto_df.tail(300)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
878,IrishCoin,Scrypt,True,PoW,4.548305e+07,64000000
879,Trollcoin,Scrypt,True,PoW/PoS,5.910227e+08,900000000
880,Litecoin Plus,Scrypt,True,PoW/PoS,2.504486e+06,4000000
881,TheGCCcoin,X13,True,PoW/PoS,NaN,2400000000
882,Monkey Project,X11,True,PoS,4.407252e+06,21000000
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [84]:
# Keep only cryptocurrencies with a working algorithm
#Does this mean no coins mined?

In [85]:
# Remove the "IsTrading" column
crypto_df.drop(['IsTrading'], axis = 1, inplace = True)

In [86]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head(300)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
560,EmberCoin,X13,PoW/PoS,9.219282e+10,850000000
563,XenixCoin,X11,PoW/PoS,3.853327e+06,3853326.77707314
565,FreeCoin,PoS,PoS,5.000000e+07,50000000
566,NPCcoin,SHA-256D,PoW/PoS,0.000000e+00,7000000000


In [87]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.drop(crypto_df.index[crypto_df['TotalCoinsMined'] == 0 ], inplace = True)
crypto_df.head(1000)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [88]:
# Drop rows where there are 'N/A' text values
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [89]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(crypto_df["CoinName"])
coins_name.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [90]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop("CoinName", axis=1, inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [91]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df)
X.head()

,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [92]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [93]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [94]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(
data = crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)    
df_crypto_pca.head(10)

,PC 1,PC 2,PC 3
0,-0.377806,-0.100950,0.018887
2,-0.358910,-0.112672,0.016359
5,0.193952,0.159577,0.076324
7,-0.060231,-0.208716,-0.166642
8,-0.045088,-0.280742,-0.116603
9,-0.197797,-0.321591,-0.152613
10,-0.347090,0.085887,0.045498
11,-0.060387,-0.217120,-0.228995
12,-0.143016,-0.285272,-0.123896
13,-0.116351,-0.215548,-0.218047


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [95]:
inertia = []
kvalue = list(range(1, 11))

for i in kvalue:
    kmeans = KMeans(n_clusters=i, random_state=0)
    kmeans.fit(df_crypto_pca)
    inertia.append(kmeans.inertia_)

elbow_plot = {"k": kvalue, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_plot)
df_elbow.hvplot.line(x="k", y="inertia", xticks=kvalue, title="elbow curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [96]:
# Initialize the K-Means model
kmodel = KMeans(n_clusters=3, random_state=0)

# Fit the model
kmodel.fit(df_crypto_pca)

# Predict clusters
predict = kmodel.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["class"] = kmodel.labels_

#Create a new DataFrame named clustered_df, that includes the following columns "Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply", "PC 1", "PC 2", "PC 3", "CoinName", "Class". You should maintain the index of the crypto_df DataFrames as is shown bellow.
clustered_df = pd.concat([crypto_df, df_crypto_pca, coins_name], axis = 1)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
0,Scrypt,PoW/PoS,4.199995e+01,42,-0.377806,-0.100950,0.018887,0,42 Coin
2,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.358910,-0.112672,0.016359,0,404Coin
5,X13,PoW/PoS,2.927942e+10,314159265359,0.193952,0.159577,0.076324,0,EliteCoin
7,SHA-256,PoW,1.792718e+07,21000000,-0.060231,-0.208716,-0.166642,0,Bitcoin
8,Ethash,PoW,1.076842e+08,0,-0.045088,-0.280742,-0.116603,0,Ethereum


### Visualizing Results

#### 3D-Scatter with Clusters

In [97]:
# Create a 3D-Scatter with the PCA data and the clusters
crypto_fig = px.scatter_3d(clustered_df,x="TotalCoinsMined",y="TotalCoinSupply",z="ProofType",width=800)
crypto_fig.update_layout(legend=dict(x=0, y=1))
crypto_fig.show()


#### Table of Tradable Cryptocurrencies

In [98]:
# Table with tradable cryptos
clustered_df.hvplot.table(width=800)

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName]

In [99]:
# Print the total number of tradable cryptocurrencies
tradable = clustered_df.shape[0]
print(tradable)

533


#### Scatter Plot with Tradable Cryptocurrencies

In [100]:
# Scale data to create the scatter plot
scaler = MinMaxScaler()
columns = ['TotalCoinsMined', 'TotalCoinSupply']
x = crypto_df[columns].values
df_scaled = scaler.fit_transform(x)
clustered_df = pd.DataFrame(df_scaled, columns=columns, index=crypto_df.index).reset_index()

In [101]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply",hover_cols=["CoinName"],)

:Scatter   [TotalCoinsMined]   (TotalCoinSupply)